In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from shapely import wkt
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score

In [2]:
train = pd.read_csv('data\Train.csv')

<>:1: SyntaxWarning: invalid escape sequence '\T'
<>:1: SyntaxWarning: invalid escape sequence '\T'
C:\Users\Aduragbemi\AppData\Local\Temp\ipykernel_23392\1100633172.py:1: SyntaxWarning: invalid escape sequence '\T'
  train = pd.read_csv('data\Train.csv')


In [3]:
train.head(3)

,FarmID,category,Crop,State,District,Sub-District,SDate,HDate,CropCoveredArea,CHeight,CNext,CLast,CTransp,IrriType,IrriSource,IrriCount,WaterCov,ExpYield,Season,geometry
0,1326576,Healthy,Paddy,Telangana,Medak,Kulcharam,2023-11-25 0:00:00,2024-04-14 0:00:00,97,54,Pea,Lentil,Transplanting,Flood,Groundwater,4,87,17,Rabi,POLYGON ((78.18143246076087 17.978884603571068...
1,1326577,Healthy,Paddy,Telangana,Medak,Kulcharam,2023-11-13 0:00:00,2024-04-26 0:00:00,82,58,Pea,Lentil,Transplanting,Flood,Canal,5,94,15,Rabi,POLYGON ((78.17545177547453 17.981074445477304...
2,1326578,Healthy,Paddy,Telangana,Medak,Kulcharam,2023-12-19 0:00:00,2024-04-28 0:00:00,92,91,Pea,Lentil,Transplanting,Flood,Canal,3,99,20,Rabi,POLYGON ((78.16914224770792 17.976212849967755...


In [4]:
test = pd.read_csv('data\Test.csv')

<>:1: SyntaxWarning: invalid escape sequence '\T'
<>:1: SyntaxWarning: invalid escape sequence '\T'
C:\Users\Aduragbemi\AppData\Local\Temp\ipykernel_23392\3210526593.py:1: SyntaxWarning: invalid escape sequence '\T'
  test = pd.read_csv('data\Test.csv')


In [5]:
test.head(3)

,FarmID,Crop,State,District,Sub-District,SDate,HDate,CropCoveredArea,CHeight,CNext,CLast,CTransp,IrriType,IrriSource,IrriCount,WaterCov,ExpYield,Season,geometry
0,85197,Paddy,Telangana,Medak,Nizampet,2023-11-12 0:00:00,2024-05-14 0:00:00,81,99,Lentil,Pea,Transplanting,Flood,Groundwater,5,81,19,Rabi,POLYGON ((78.54742059207287 18.062307326758276...
1,779677,Paddy,Telangana,Medak,Nizampet,2023-12-20 0:00:00,2024-05-28 0:00:00,91,63,Lentil,Pea,Transplanting,Flood,Canal,5,97,15,Rabi,"POLYGON ((78.54812514990353 18.06739449795583,..."
2,1331840,Paddy,Telangana,Medak,Nizampet,2023-11-29 0:00:00,2024-05-18 0:00:00,94,89,Pea,Lentil,Transplanting,Flood,Canal,5,88,15,Rabi,POLYGON ((78.56002792938853 18.079607077817514...


### **SET _FarmID_ AS THE INDEX**

In [6]:
train.set_index('FarmID', inplace=True)

In [7]:
test.set_index('FarmID', inplace=True)

### **CONVERT *SDate* AND *HDate* FEATURE TO DATETIME**

In [8]:
train['SDate'] = pd.to_datetime(train['SDate'])
train['HDate'] = pd.to_datetime(train['HDate']) 

print(train['SDate'].dtype)
print(train['HDate'].dtype)

datetime64[ns]
datetime64[ns]


In [9]:
test['SDate'] = pd.to_datetime(test['SDate'])
test['HDate'] = pd.to_datetime(test['HDate'])

print(test['SDate'].dtype)
print(test['HDate'].dtype)

datetime64[ns]
datetime64[ns]


### **EXTRACTING FEATURES FROM _geometry_**

In [10]:
train['geometry'] = train['geometry'].astype(str).apply(wkt.loads)  
train['Area'] = train['geometry'].apply(lambda x: x.area if x else None)
train['Perimeter'] = train['geometry'].apply(lambda x: x.length if x else None)   

# Calculate centroid
train['Centroid'] = train['geometry'].apply(lambda x: x.centroid if x else None)

# Optionally split centroid into latitude and longitude
train['Centroid_Lat'] = train['Centroid'].apply(lambda x: x.y if x else None)
train['Centroid_Lon'] = train['Centroid'].apply(lambda x: x.x if x else None)

train = train.drop(['geometry', 'Centroid'], axis=1)


In [11]:
test['geometry'] = test['geometry'].astype(str).apply(wkt.loads)  
test['Area'] = test['geometry'].apply(lambda x: x.area if x else None)
test['Perimeter'] = test['geometry'].apply(lambda x: x.length if x else None)   

# Calculate centroid
test['Centroid'] = test['geometry'].apply(lambda x: x.centroid if x else None)

# Optionally split centroid into latitude and longitude
test['Centroid_Lat'] = test['Centroid'].apply(lambda x: x.y if x else None)
test['Centroid_Lon'] = test['Centroid'].apply(lambda x: x.x if x else None)

test = test.drop(['geometry', 'Centroid'], axis=1)

### **EXTRACT *day*, *month* AND *week_of_month* FROM *SDate* AND *HDate***

In [12]:
from datetime import datetime

def week_of_month(dt):
    """Calculate the week of the month for a given datetime object."""
    first_day = dt.replace(day=1)
    dom = dt.day  # Day of the month
    adjusted_dom = dom + first_day.weekday()  # Adjust for the first week's weekday
    return (adjusted_dom - 1) // 7 + 1

In [13]:
train['SDay'] = train['SDate'].dt.day
train['SMonth'] = train['SDate'].dt.month
train['SWOM'] = train['SDate'].apply(week_of_month)
train['SYear'] = train['SDate'].dt.year

train['HDay'] = train['HDate'].dt.day
train['HMonth'] = train['HDate'].dt.month
train['HWOM'] = train['HDate'].apply(week_of_month)
train['HYear'] = train['HDate'].dt.year

train.head(3)

,category,Crop,State,District,Sub-District,SDate,HDate,CropCoveredArea,CHeight,CNext,...,Centroid_Lat,Centroid_Lon,SDay,SMonth,SWOM,SYear,HDay,HMonth,HWOM,HYear
FarmID,,,,,,,,,,,,,,,,,,,,,
1326576,Healthy,Paddy,Telangana,Medak,Kulcharam,2023-11-25,2024-04-14,97,54,Pea,...,17.978863,78.181592,25,11,4,2023,14,4,2,2024
1326577,Healthy,Paddy,Telangana,Medak,Kulcharam,2023-11-13,2024-04-26,82,58,Pea,...,17.980966,78.175606,13,11,3,2023,26,4,4,2024
1326578,Healthy,Paddy,Telangana,Medak,Kulcharam,2023-12-19,2024-04-28,92,91,Pea,...,17.976084,78.169207,19,12,4,2023,28,4,4,2024


In [14]:
test['SDay'] = test['SDate'].dt.day
test['SMonth'] = test['SDate'].dt.month
test['SWOM'] = test['SDate'].apply(week_of_month)
test['SYear'] = test['SDate'].dt.year

test['HDay'] = test['HDate'].dt.day
test['HMonth'] = test['HDate'].dt.month
test['HWOM'] = test['HDate'].apply(week_of_month)
test['HYear'] = test['HDate'].dt.year

test.head(3)

,Crop,State,District,Sub-District,SDate,HDate,CropCoveredArea,CHeight,CNext,CLast,...,Centroid_Lat,Centroid_Lon,SDay,SMonth,SWOM,SYear,HDay,HMonth,HWOM,HYear
FarmID,,,,,,,,,,,,,,,,,,,,,
85197,Paddy,Telangana,Medak,Nizampet,2023-11-12,2024-05-14,81,99,Lentil,Pea,...,18.062054,78.547474,12,11,2,2023,14,5,3,2024
779677,Paddy,Telangana,Medak,Nizampet,2023-12-20,2024-05-28,91,63,Lentil,Pea,...,18.067696,78.548407,20,12,4,2023,28,5,5,2024
1331840,Paddy,Telangana,Medak,Nizampet,2023-11-29,2024-05-18,94,89,Pea,Lentil,...,18.079480,78.560171,29,11,5,2023,18,5,3,2024


### **GROWING SEASON DURATION**

In [15]:
train['SeasonDuration'] = (train['HDate'] - train['SDate']).dt.days
test['SeasonDuration'] = (test['HDate'] - test['SDate']).dt.days


train = train.drop(['SDate', 'HDate'], axis=1)
test = test.drop(['SDate', 'HDate'], axis=1)

### **SEASONAL INDICATORS**

[Know India](https://knowindia.india.gov.in/profile/climate.php#:~:text=Winter%2C%20from%20December%20to%20early,monsoon%2C%20from%20October%20to%20December)

In [16]:
# Kharif and Rabi seasons experience the most harvest in India
# Kharif (Rainy): June to September
# Rabi (Winter): December to April


train['Rainy_Winter'] = train['SMonth'].apply(lambda x: 1 if x in [6, 7, 8, 9, 12, 1, 2, 3, 4] else 0)
test['Rainy_Winter'] = test['SMonth'].apply(lambda x: 1 if x in [6, 7, 8, 9, 12, 1, 2, 3, 4] else 0)

### **CROP IRRIGATION TYPE METHOD**

In [17]:
# Interaction Features
train['CropIrriType'] = train['Crop'].astype(str) + "_" + train['IrriType'].astype(str)
test['CropIrriType'] = test['Crop'].astype(str) + "_" + test['IrriType'].astype(str)

### **CROP IRRIGATION SOURCE METHOD**

In [18]:
train['CropIrriSource'] = train['Crop'].astype(str) + "_" + train['IrriSource'].astype(str)
test['CropIrriSource'] = test['Crop'].astype(str) + "_" + test['IrriSource'].astype(str)

### **DROP *State* FEATURE**

In [19]:
train = train.drop(['State'], axis=1)
test = test.drop(['State'], axis=1)

In [20]:
train[['Area', 'Perimeter']].iloc[0]

Area         3.982611e-08
Perimeter    8.487694e-04
Name: 1326576, dtype: float64

### **SCALE _Area_ AND _Perimeter_**

In [21]:
train['Area'] = train['Area'] * 20000000
train['Perimeter'] = train['Perimeter'] * 2000000

In [22]:
test['Area'] = test['Area'] * 20000000
test['Perimeter'] = test['Perimeter'] * 2000000

### **CHANGE DATA TYPE OF NON-NUMERIC FEATURE TO _Category_**

In [23]:
train_cat_cols = train.select_dtypes(include='object').columns

for col in train_cat_cols:
    train[col] = train[col].astype('category')

train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7888 entries, 1326576 to 1330504
Data columns (total 31 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   category         7888 non-null   category
 1   Crop             7888 non-null   category
 2   District         7888 non-null   category
 3   Sub-District     7888 non-null   category
 4   CropCoveredArea  7888 non-null   int64   
 5   CHeight          7888 non-null   int64   
 6   CNext            7888 non-null   category
 7   CLast            7888 non-null   category
 8   CTransp          7888 non-null   category
 9   IrriType         7888 non-null   category
 10  IrriSource       7888 non-null   category
 11  IrriCount        7888 non-null   int64   
 12  WaterCov         7888 non-null   int64   
 13  ExpYield         7888 non-null   int64   
 14  Season           7888 non-null   category
 15  Area             7888 non-null   float64 
 16  Perimeter        7888 non-null   float

In [24]:
test_cat_cols = test.select_dtypes(include='object').columns

for col in test_cat_cols:
    test[col] = test[col].astype('category')

test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2718 entries, 85197 to 1326552
Data columns (total 30 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   Crop             2718 non-null   category
 1   District         2718 non-null   category
 2   Sub-District     2718 non-null   category
 3   CropCoveredArea  2718 non-null   int64   
 4   CHeight          2718 non-null   int64   
 5   CNext            2718 non-null   category
 6   CLast            2718 non-null   category
 7   CTransp          2718 non-null   category
 8   IrriType         2718 non-null   category
 9   IrriSource       2718 non-null   category
 10  IrriCount        2718 non-null   int64   
 11  WaterCov         2718 non-null   int64   
 12  ExpYield         2718 non-null   int64   
 13  Season           2718 non-null   category
 14  Area             2718 non-null   float64 
 15  Perimeter        2718 non-null   float64 
 16  Centroid_Lat     2718 non-null   float64

### **ENCODE _Categorical_ FEATURES**

In [25]:
for col in train_cat_cols:
    train[col] = train[col].cat.codes

train.head(3)

,category,Crop,District,Sub-District,CropCoveredArea,CHeight,CNext,CLast,CTransp,IrriType,...,SWOM,SYear,HDay,HMonth,HWOM,HYear,SeasonDuration,Rainy_Winter,CropIrriType,CropIrriSource
FarmID,,,,,,,,,,,,,,,,,,,,,
1326576,1,5,5,61,97,54,4,0,3,1,...,4,2023,14,4,2,2024,141,0,10,16
1326577,1,5,5,61,82,58,4,0,3,1,...,3,2023,26,4,4,2024,165,0,10,15
1326578,1,5,5,61,92,91,4,0,3,1,...,4,2023,28,4,4,2024,131,1,10,15


In [26]:
for col in test_cat_cols:
    test[col] = test[col].cat.codes

test.head(3)

,Crop,District,Sub-District,CropCoveredArea,CHeight,CNext,CLast,CTransp,IrriType,IrriSource,...,SWOM,SYear,HDay,HMonth,HWOM,HYear,SeasonDuration,Rainy_Winter,CropIrriType,CropIrriSource
FarmID,,,,,,,,,,,,,,,,,,,,,
85197,5,5,25,81,99,0,4,3,1,1,...,2,2023,14,5,3,2024,184,0,10,16
779677,5,5,25,91,63,0,4,3,1,0,...,4,2023,28,5,5,2024,160,1,10,15
1331840,5,5,25,94,89,4,0,3,1,0,...,5,2023,18,5,3,2024,171,0,10,15


### **MODEL BUILDING**

In [27]:
X = train.drop(['category'], axis=1)
y = train['category']

In [48]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [49]:
# Oversampling the minority class
sm = SMOTE(random_state=42)
X_train, y_train = sm.fit_resample(X_train, y_train)

### **SCALING**

In [50]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

### **BUILDING WITH XGBOOST**

In [51]:
model = xgb.XGBClassifier(objective='multi:softmax', 
                          booster='gbtree',
                          num_class=4, 
                          enable_categorical=True, 
                          n_estimators=500,
                          random_state=42)

In [52]:
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=True, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=500, n_jobs=None, num_class=4,
              num_parallel_tree=None, ...)

In [53]:
y_pred = model.predict(X_test)

In [54]:
f1_score(y_test, y_pred, average='weighted')

0.7415122432417766

### **BUILDING WITH CATBOOST**

In [55]:
from catboost import CatBoostClassifier

In [56]:
catmodel = CatBoostClassifier(iterations=500, 
                              learning_rate=0.3, 
                              depth=6, 
                              loss_function='MultiClass', 
                              random_state=42, 
                              class_weights=[1, 4, 4, 4])

In [57]:
catmodel.fit(X_train, y_train)

0:	learn: 1.2930633	total: 170ms	remaining: 1m 24s
1:	learn: 1.2200638	total: 201ms	remaining: 50s
2:	learn: 1.1735255	total: 236ms	remaining: 39.1s
3:	learn: 1.1435868	total: 269ms	remaining: 33.4s
4:	learn: 1.1243140	total: 301ms	remaining: 29.8s
5:	learn: 1.0966548	total: 390ms	remaining: 32.1s
6:	learn: 1.0709620	total: 432ms	remaining: 30.4s
7:	learn: 1.0546331	total: 466ms	remaining: 28.7s
8:	learn: 1.0403522	total: 499ms	remaining: 27.2s
9:	learn: 1.0282243	total: 526ms	remaining: 25.8s
10:	learn: 1.0118240	total: 565ms	remaining: 25.1s
11:	learn: 0.9957839	total: 614ms	remaining: 25s
12:	learn: 0.9832972	total: 658ms	remaining: 24.6s
13:	learn: 0.9702851	total: 711ms	remaining: 24.7s
14:	learn: 0.9626051	total: 843ms	remaining: 27.3s
15:	learn: 0.9490825	total: 971ms	remaining: 29.4s
16:	learn: 0.9427057	total: 1.03s	remaining: 29.2s
17:	learn: 0.9343519	total: 1.11s	remaining: 29.7s
18:	learn: 0.9269986	total: 1.23s	remaining: 31.2s
19:	learn: 0.9214392	total: 1.3s	remaining: 

In [58]:
catpred = catmodel.predict(X_test) 

In [59]:
f1_score(y_test, catpred, average='weighted')

0.7411980472324521

### **USING LIGHTGBM**

In [60]:
import lightgbm as lgb

In [61]:
light_model = lgb.LGBMClassifier(objective='multiclass', 
                                 random_state=42, 
                                 n_estimators=500,
                                 learning_rate=0.3)
light_model.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002837 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2013
[LightGBM] [Info] Number of data points in the train set: 20804, number of used features: 30
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Start training from score -1.386294


LGBMClassifier(learning_rate=0.3, n_estimators=500, objective='multiclass',
               random_state=42)

In [62]:
light_pred = light_model.predict(X_test)

In [63]:
f1_score(y_test, light_pred, average='weighted')

0.7407700719856433

### **USING RANDOMFORESTCLASSIFIER**

In [64]:
clf = RandomForestClassifier(n_estimators=500, 
                             class_weight='balanced', 
                             random_state=42)

In [65]:
clf.fit(X_train, y_train)

RandomForestClassifier(max_depth=28, max_features='log2', min_samples_split=8,
                       n_estimators=131, random_state=42)

In [66]:
clf_pred = clf.predict(X_test)

In [67]:
f1_score(y_test, clf_pred, average='weighted')

0.7465027027765206

### **PREDICTION ON TEST SET**

In [48]:
test_pred = light_model.predict(test)
test_pred

array([1, 1, 1, ..., 1, 1, 1], dtype=int8)

In [49]:
submission = pd.DataFrame(
    {
        'ID': test.index,
        'Target': test_pred
    }
)

In [50]:
category_map = {0: 'Diseased', 1: 'Healthy', 2: 'Pests', 3: 'Stressed'}

submission['Target'] = submission['Target'].map(category_map)
submission.head(3)

,ID,Target
0,85197,Healthy
1,779677,Healthy
2,1331840,Healthy


In [51]:
submission.to_csv('submission2.csv', index=False)